In [7]:
# dependencies
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import ComplementNB, MultinomialNB
import numpy as np
from scipy.stats import uniform, norm
from sklearn.metrics import plot_confusion_matrix
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn import svm
from XLB import *

import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn import datasets

import warnings
warnings.filterwarnings('ignore')

In [8]:
# extract data from files
x_train, y_train = extract_data("FinalTrainingSet.csv")
x_val, y_val = extract_data("Validation Set.csv")

#scale data values
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

# feature selection
num_features = 45
feat_sel = VarianceThreshold()
x_train = feat_sel.fit_transform(x_train)
# feat_sel_2 = SelectFromModel(estimator=DecisionTreeClassifier(random_state=481516234))
# feat_sel_2 = SelectFromModel(\
#                 estimator=RandomForestClassifier(n_estimators=50,\
#                                              random_state=481516234))
# 33.33% 0.24
feat_sel_2 = SelectFromModel(\
                estimator=LogisticRegression(random_state=481516234))
# feat_sel_2 = SelectFromModel(\
#                 estimator=svm.LinearSVC(C=0.25, penalty="l1", dual=False,\
#                                     random_state=481516234))
# feat_sel_2 = SelectKBest(mutual_info_classif,k=num_features)
x_train = feat_sel_2.fit_transform(x_train,y_train)
x_val = feat_sel_2.transform(feat_sel.transform(x_val))
# print("After Variance Threshold Feature Selection:",x_train.shape)

rand_seed = 3454132

oversampler = SMOTE(sampling_strategy="not majority",random_state=rand_seed)
x_smote, y_smote = oversampler.fit_resample(x_train,y_train)
print(x_smote.shape,y_smote.shape)

oversampler = RandomOverSampler(sampling_strategy="not majority",random_state=rand_seed)
x_os, y_os = oversampler.fit_resample(x_train,y_train)
print(x_os.shape,y_os.shape)

pd.cut(x_train[1], bins=3, labels=np.arange(3), right=False)

print(x_train[1])

(750, 69) (750,)
(750, 69) (750,)
[1.         1.         1.         0.52108434 0.48107515 0.72424866
 0.43380062 0.52308697 0.52817024 0.12486843 0.54195949 0.3611159
 0.52129297 0.27518074 0.46655604 0.6557498  0.30235053 0.32643465
 0.02736892 0.55518878 0.59222226 0.09831908 0.54163232 0.36127085
 0.52209661 0.27414381 0.46502436 0.6529655  0.30123643 0.72383199
 0.65381368 0.4523244  0.13277089 0.18917784 0.09148758 0.50564972
 0.77948653 0.68639304 0.09099316 0.01541819 0.71095813 0.7826087
 0.08241345 0.77948653 0.6864991  0.0910664  0.0145165  0.70895515
 0.77009661 0.67647059 0.00203913 0.00562726 0.01349145 0.01986507
 0.0123881  0.01746491 0.0123881  0.01746491 0.00204347 0.00454104
 0.00204347 0.00454104 0.00284424 0.25971479 0.3756087  0.20311342
 0.23618019 0.17934551 0.1793323 ]


In [9]:
themes = ["Calm","Cheerful","Bravery","Fearful","Love","Sadness"]
ovr_train = []
ovr_val = []
ovr_y_smote = []
ovr_y_os = []
for i in range(1,7):
    ovr_train.append(ovr_labels(y_train, i))
    ovr_val.append(ovr_labels(y_val,i))
    ovr_y_os.append(ovr_labels(y_os,i))
    ovr_y_smote.append(ovr_labels(y_smote,i))

In [10]:
class AbstractCustomModel:
#     """
#     This is the default constructor for this class.
 
#     Parameters:
#     params : dict - hyperparameters for the model
#     """
    def __init__(self,params):
        self.params = params

#     This method trains the model on the given dataset
 
#     Parameters:
#     x_train : numpy.ndarray - training set data
#     """
    def train(self,x_train,y_train=None):
        # TODO
        pass

#     This method evaluates the performance of the model on the given test set.
 
#     Parameters:
#     x_test : numpy.ndarray - test set data
 
#     Returns:
#     A floating point number which measures the performance of the model on the 
#     test set.
#     """

    def evaluate(self,x_test,y_test=None):
        # TODO
        pass

In [1]:
# """
# This class is an adapter for the Bayesian Network library.
# """
class BayesianNetworkAdapter(AbstractCustomModel):
    
#  This is the default constructor for this class.
 
#     Parameters:
#     params : dict - hyperparameters for the model
#     """
    def __init__(self,params):
        self.params = params
        super().__init__()
 
 
#     Parameters:
#     x_train : numpy.ndarray - training set data

    def train(self,x_train,y_train=None):
        #DATA IS X_TRAIN
        data = x_train
        # target = np.reshape(np.array(list(map(int,y_train))), (x_train.shape[0], 1))
        target = y_train-1
        # print(target.size)
        print(y_train-1)
        data_tensor= torch.from_numpy(data).float()
        target_tensor= torch.from_numpy(target).long()

        #Building the model
        # prior_mu (Float) is the mean of prior normal distribution.
        # prior_sigma (Float) is the sigma of prior normal distribution.
        model = nn.Sequential(
            bnn.BayesLinear(prior_mu=0.5, prior_sigma=0.15, in_features=69, out_features=100),
            nn.ReLU(),
            bnn.BayesLinear(prior_mu=0.5, prior_sigma=0.15, in_features=100, out_features=6),
        )

        #The two-loss functions used here are cross-entropy loss and the BKL loss
        #which is used to compute the KL (Kullback–Leibler) divergence of the network.
        cross_entropy_loss = nn.CrossEntropyLoss()
        klloss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
        klweight = 0.01
        optimizer = optim.Adam(model.parameters(), lr=3e-4)

        #TRAINING
        for step in range(500):
            models = model(data_tensor)
        #     print(models.size(0))
            cross_entropy = cross_entropy_loss(models, target_tensor)
            kl = klloss(model)
            total_cost = cross_entropy + klweight*kl

            optimizer.zero_grad()
            total_cost.backward()
            optimizer.step()

        _, predicted = torch.max(models.data, 1)
        final = target_tensor.size(0)
        correct = (predicted == target_tensor).sum()
        print('- Accuracy: %f %%' % (100 * float(correct) / final))
        print('- CE : %2.2f, KL : %2.2f' % (cross_entropy.item(), kl.item()))
        
        pass
 
   #   This method evaluates the performance of the model on the given test set.
#     Parameters:
#     x_test : numpy.ndarray - test set data
 
#     Returns:
#     A floating point number which measures the performance of the model on the 
#     test set.
#     """
    def evaluate(self,x_test,y_test=None):
        # TODO
        pass
 


NameError: name 'AbstractCustomModel' is not defined

In [29]:
# dataset = datasets.load_iris()

#DATA IS X_TRAIN
data = x_train
# target = np.reshape(np.array(list(map(int,y_train))), (x_train.shape[0], 1))
target = y_train-1
# print(target.size)
print(y_train-1)
data_tensor= torch.from_numpy(data).float()
target_tensor= torch.from_numpy(target).long()

#Building the model
# prior_mu (Float) is the mean of prior normal distribution.
# prior_sigma (Float) is the sigma of prior normal distribution.
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0.5, prior_sigma=0.15, in_features=69, out_features=100),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0.5, prior_sigma=0.15, in_features=100, out_features=6),
)

#The two-loss functions used here are cross-entropy loss and the BKL loss
#which is used to compute the KL (Kullback–Leibler) divergence of the network.
cross_entropy_loss = nn.CrossEntropyLoss()
klloss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
klweight = 0.01
optimizer = optim.Adam(model.parameters(), lr=3e-4)

#TRAINING
for step in range(500):
    models = model(data_tensor)
#     print(models.size(0))
    cross_entropy = cross_entropy_loss(models, target_tensor)
    kl = klloss(model)
    total_cost = cross_entropy + klweight*kl

    optimizer.zero_grad()
    total_cost.backward()
    optimizer.step()
  
_, predicted = torch.max(models.data, 1)
final = target_tensor.size(0)
correct = (predicted == target_tensor).sum()
print('- Accuracy: %f %%' % (100 * float(correct) / final))
print('- CE : %2.2f, KL : %2.2f' % (cross_entropy.item(), kl.item()))


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 3. 0. 3. 0. 0. 0. 0. 2. 3. 5. 0. 2. 2. 5.
 0. 0. 0. 2. 0. 3. 0. 0. 2. 2. 2. 0. 2. 2. 2. 0. 0. 0. 3. 2. 1. 0. 1. 0.
 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 1. 3. 3. 2. 2. 4. 0. 0. 1. 1. 0. 0. 5. 0.
 3. 0. 1. 0. 3. 0. 5. 0. 0. 2. 0. 0. 3. 2. 2. 3. 5.

In [20]:
from sklearn import datasets

dataset = datasets.load_iris()
print(dataset.target.size(0))


print(y_train)

TypeError: 'int' object is not callable